# Pre-processing Colorado Site Specific data for WaDEQA upload.
Date Updated: 03/09/2021
Purpose:  To pre-process the Colorado data into one master file for simple DataFrame creation and extraction

Notes:
- Using two different API Colorado CDSS REST web service. 1) [**Division Data**](https://dwr.state.co.us/Rest/GET/Help/Api/GET-api-v2-structures-divrec-waterclasses) api for Division 1-7 site specific information. 2) [**Annual WDID Time Series Data**](https://dwr.state.co.us/Rest/GET/Help/Api/GET-api-v2-structures-divrec-divrecyear) api using sites of interest wdid list produced from Division 1-7.
- Retreiving WDID data that is divrectype = WaterClass, availableTimesteps = Year, and ciuCode = A (for active) values only.
- Full script run time took the better of the morning.  Will save each division incremental step as individiual csv output for ease of review.

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
import requests
import json
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [2]:
#Working Directory and Input File
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Colorado/ObservationSite/RawInputData"
os.chdir(workingDir)

In [ ]:
# Query by Division (1-7) to get a full list of WDIDs per division.
# Plug in "division=1" etc into API request.
# Save results as Division1.csv, etc.
# Rinse and Repeat to retreive all data for Divisions 1-7.

# url = "https://dwr.state.co.us/Rest/GET/api/v2/structures/divrec/waterclasses/?division=7&apiKey=wAC6ZmzcPJ30dyy6nYu6jQmG7BBedcem"
# responseD = json.loads(requests.get(url).text)
# LD = responseD['ResultList']

# df_ts = pd.DataFrame()
# for n in range(len(LD)):
#     row = pd.DataFrame([LD[n]])
#     df_ts = df_ts.append(row)
# df_ts

# #Exporting to Finished File
# df_ts.to_csv('Division7.csv', index=False)  # The output

# Division 1

In [ ]:
# The Dvision 1 Time Series Dataframe
df_div1ts = pd.DataFrame()

In [ ]:
# Read in division csv file.
# Change input to Division 1
fileInput = "Success/Division1.csv"
dftemp = pd.read_csv(fileInput)
print(len(dftemp))
dftemp.head(3)

In [ ]:
# Trim down division dataframe for divrectype = DivTotal, availableTimesteps = Year, and ciuCode = A (for active) values only.
# Fix wdid values that are less then 7 chars long.  Covert to string.
dftemp = dftemp[dftemp.divrectype == 'DivTotal']
dftemp = dftemp[dftemp.availableTimesteps == 'Year']
dftemp = dftemp[dftemp.ciuCode == 'A']

# fix wdid values that are less then 7 chars long.  Covert to string.
def formatWDIDValue(colVal):
    n = str(colVal)
    if len(n) < 7:
        outString = "0" + n
    else:
        outString = n
    return outString
dftemp['wdid'] = dftemp.apply(lambda row: formatWDIDValue(row['wdid']), axis=1)

print(len(dftemp))
dftemp.head(3)

In [ ]:
# Converted trimed down dataframe wdid to list.
dftemp = dftemp.drop_duplicates(subset='wdid', keep="first")
wdidlist = dftemp['wdid'].tolist()
type(wdidlist)

In [ ]:
#Split list into catagories that are 100 long.
wdidlistB = [wdidlist[i:i + 100] for i in range(0, len(wdidlist), 100)]

In [ ]:
# Use list of WDIDs (from Divisoin data) as inputs, retreive time series data.
# Change dataframe to df_div1ts

str2 = "https://dwr.state.co.us/Rest/GET/api/v2/structures/divrec/divrecyear/?wdid="
str3 = "%2C&apiKey=wAC6ZmzcPJ30dyy6nYu6jQmG7BBedcem"

for i in range(len(wdidlistB)):
    lstC = wdidlistB[i]
    lstCa = '%2C'.join([str(n) for n in lstC]) 
    
    url = str2 + lstCa + str3
    responseD = json.loads(requests.get(url).text)
    LD = responseD['ResultList']
    
    for n in range(len(LD)):
        row = pd.DataFrame([LD[n]])
        df_div1ts = df_div1ts.append(row)

print(len(df_div1ts))
df_div1ts.tail(3)

In [ ]:
# Merging wdid and division dataframes into one, using left-join.
df_div1ts = pd.merge(df_div1ts, dftemp, left_on='wdid', right_on='wdid', how='left') 

In [ ]:
# Exporting to Finished File.
# Change dataframe to df_div1ts.
df_div1ts.to_excel('P_Div1WDIDTimeSeries.xlsx', index=False)  # The output

# Division 2

In [ ]:
# The Dvision 2 Time Series Dataframe
df_div2ts = pd.DataFrame()

In [ ]:
# Read in division csv file.
# Change input to Division 2
fileInput = "Success/Division2.csv"
dftemp = pd.read_csv(fileInput)
print(len(dftemp))
dftemp.head(3)

In [ ]:
# Trim down division dataframe for divrectype = DivTotal, availableTimesteps = Year, and ciuCode = A (for active) values only.
# Fix wdid values that are less then 7 chars long.  Covert to string.
dftemp = dftemp[dftemp.divrectype == 'DivTotal']
dftemp = dftemp[dftemp.availableTimesteps == 'Year']
dftemp = dftemp[dftemp.ciuCode == 'A']

# fix wdid values that are less then 7 chars long.  Covert to string.
def formatWDIDValue(colVal):
    n = str(colVal)
    if len(n) < 7:
        outString = "0" + n
    else:
        outString = n
    return outString
dftemp['wdid'] = dftemp.apply(lambda row: formatWDIDValue(row['wdid']), axis=1)

print(len(dftemp))
dftemp.head(3)

In [ ]:
# Converted trimed down dataframe wdid to list.
dftemp = dftemp.drop_duplicates(subset='wdid', keep="first")
wdidlist = dftemp['wdid'].tolist()
type(wdidlist)

In [ ]:
#Split list into catagories that are 100 long.
wdidlistB = [wdidlist[i:i + 100] for i in range(0, len(wdidlist), 100)]

In [ ]:
# Use list of WDIDs (from Divisoin data) as inputs, retreive time series data.
# Change dataframe to df_div2ts

str2 = "https://dwr.state.co.us/Rest/GET/api/v2/structures/divrec/divrecyear/?wdid="
str3 = "%2C&apiKey=wAC6ZmzcPJ30dyy6nYu6jQmG7BBedcem"

for i in range(len(wdidlistB)):
    lstC = wdidlistB[i]
    lstCa = '%2C'.join([str(n) for n in lstC]) 
    
    url = str2 + lstCa + str3
    responseD = json.loads(requests.get(url).text)
    LD = responseD['ResultList']
    
    for n in range(len(LD)):
        row = pd.DataFrame([LD[n]])
        df_div2ts = df_div2ts.append(row)

print(len(df_div2ts))
df_div2ts.tail(3)

In [ ]:
# Merging wdid and division dataframes into one, using left-join.
df_div2ts = pd.merge(df_div2ts, dftemp, left_on='wdid', right_on='wdid', how='left')

In [ ]:
# Exporting to Finished File.
# Change dataframe to df_div2ts.
df_div2ts.to_excel('P_Div2WDIDTimeSeries.xlsx', index=False)  # The output

# Division 3

In [ ]:
# The Dvision 3 Time Series Dataframe
df_div3ts = pd.DataFrame()

In [ ]:
# Read in division csv file.
# Change input to Division 3.
fileInput = "Success/Division3.csv"
dftemp = pd.read_csv(fileInput)
print(len(dftemp))
dftemp.head(3)

In [ ]:
# Trim down division dataframe for divrectype = DivTotal, availableTimesteps = Year, and ciuCode = A (for active) values only.
# Fix wdid values that are less then 7 chars long.  Covert to string.
dftemp = dftemp[dftemp.divrectype == 'DivTotal']
dftemp = dftemp[dftemp.availableTimesteps == 'Year']
dftemp = dftemp[dftemp.ciuCode == 'A']

# fix wdid values that are less then 7 chars long.  Covert to string.
def formatWDIDValue(colVal):
    n = str(colVal)
    if len(n) < 7:
        outString = "0" + n
    else:
        outString = n
    return outString
dftemp['wdid'] = dftemp.apply(lambda row: formatWDIDValue(row['wdid']), axis=1)

print(len(dftemp))
dftemp.head(3)

In [ ]:
# Converted trimed down dataframe wdid to list.
dftemp = dftemp.drop_duplicates(subset='wdid', keep="first")
wdidlist = dftemp['wdid'].tolist()
type(wdidlist)

In [ ]:
# Split list into catagories that are 100 long.
wdidlistB = [wdidlist[i:i + 100] for i in range(0, len(wdidlist), 100)]

In [ ]:
# Use list of WDIDs (from Divisoin data) as inputs, retreive time series data.
# Change dataframe to df_div3ts

str2 = "https://dwr.state.co.us/Rest/GET/api/v2/structures/divrec/divrecyear/?wdid="
str3 = "%2C&apiKey=wAC6ZmzcPJ30dyy6nYu6jQmG7BBedcem"

for i in range(len(wdidlistB)):
    lstC = wdidlistB[i]
    lstCa = '%2C'.join([str(n) for n in lstC]) 
    
    url = str2 + lstCa + str3
    responseD = json.loads(requests.get(url).text)
    LD = responseD['ResultList']
    
    for n in range(len(LD)):
        row = pd.DataFrame([LD[n]])
        df_div3ts = df_div3ts.append(row)

print(len(df_div3ts))
df_div3ts.tail(3)

In [ ]:
# Merging wdid and division dataframes into one, using left-join.
df_div3ts = pd.merge(df_div3ts, dftemp, left_on='wdid', right_on='wdid', how='left')

In [ ]:
# Exporting to Finished File.
# Change dataframe to df_div3ts.
df_div3ts.to_excel('P_Div3WDIDTimeSeries.xlsx', index=False)  # The output

# Division 4

In [ ]:
# The Dvision 4 Time Series Dataframe
df_div4ts = pd.DataFrame()

In [ ]:
# Read in division csv file.
# Change input to Division 4.
fileInput = "Success/Division4.csv"
dftemp = pd.read_csv(fileInput)
print(len(dftemp))
dftemp.head(3)

In [ ]:
# Trim down division dataframe for divrectype = DivTotal, availableTimesteps = Year, and ciuCode = A (for active) values only.
# Fix wdid values that are less then 7 chars long.  Covert to string.
dftemp = dftemp[dftemp.divrectype == 'DivTotal']
dftemp = dftemp[dftemp.availableTimesteps == 'Year']
dftemp = dftemp[dftemp.ciuCode == 'A']

# fix wdid values that are less then 7 chars long.  Covert to string.
def formatWDIDValue(colVal):
    n = str(colVal)
    if len(n) < 7:
        outString = "0" + n
    else:
        outString = n
    return outString
dftemp['wdid'] = dftemp.apply(lambda row: formatWDIDValue(row['wdid']), axis=1)

print(len(dftemp))
dftemp.head(3)

In [ ]:
# Converted trimed down dataframe wdid to list.
dftemp = dftemp.drop_duplicates(subset='wdid', keep="first")
wdidlist = dftemp['wdid'].tolist()
type(wdidlist)

In [ ]:
# Split list into catagories that are 100 long.
wdidlistB = [wdidlist[i:i + 100] for i in range(0, len(wdidlist), 100)]

In [ ]:
# Use list of WDIDs (from Divisoin data) as inputs, retreive time series data.
# Change dataframe to df_div4ts

str2 = "https://dwr.state.co.us/Rest/GET/api/v2/structures/divrec/divrecyear/?wdid="
str3 = "%2C&apiKey=wAC6ZmzcPJ30dyy6nYu6jQmG7BBedcem"

for i in range(len(wdidlistB)):
    lstC = wdidlistB[i]
    lstCa = '%2C'.join([str(n) for n in lstC]) 
    
    url = str2 + lstCa + str3
    responseD = json.loads(requests.get(url).text)
    LD = responseD['ResultList']
    
    for n in range(len(LD)):
        row = pd.DataFrame([LD[n]])
        df_div4ts = df_div4ts.append(row)

print(len(df_div4ts))
df_div4ts.tail(3)

In [ ]:
# Merging wdid and division dataframes into one, using left-join.
df_div4ts = pd.merge(df_div4ts, dftemp, left_on='wdid', right_on='wdid', how='left')

In [ ]:
# Exporting to Finished File.
# Change dataframe to df_div4ts.
df_div4ts.to_excel('P_Div4WDIDTimeSeries.xlsx', index=False)  # The output

# Division 5

In [ ]:
# The Dvision 5 Time Series Dataframe
df_div5ts = pd.DataFrame()

In [ ]:
# Read in division csv file.
# Change input to Division 5.
fileInput = "Success/Division5.csv"
dftemp = pd.read_csv(fileInput)
print(len(dftemp))
dftemp.head(3)

In [ ]:
# Trim down division dataframe for divrectype = DivTotal, availableTimesteps = Year, and ciuCode = A (for active) values only.
# Fix wdid values that are less then 7 chars long.  Covert to string.
dftemp = dftemp[dftemp.divrectype == 'DivTotal']
dftemp = dftemp[dftemp.availableTimesteps == 'Year']
dftemp = dftemp[dftemp.ciuCode == 'A']

# fix wdid values that are less then 7 chars long.  Covert to string.
def formatWDIDValue(colVal):
    n = str(colVal)
    if len(n) < 7:
        outString = "0" + n
    else:
        outString = n
    return outString
dftemp['wdid'] = dftemp.apply(lambda row: formatWDIDValue(row['wdid']), axis=1)

print(len(dftemp))
dftemp.head(3)

In [ ]:
# Converted trimed down dataframe wdid to list.
dftemp = dftemp.drop_duplicates(subset='wdid', keep="first")
wdidlist = dftemp['wdid'].tolist()
type(wdidlist)

In [ ]:
# Split list into catagories that are 100 long.
wdidlistB = [wdidlist[i:i + 100] for i in range(0, len(wdidlist), 100)]

In [ ]:
# Use list of WDIDs (from Divisoin data) as inputs, retreive time series data.
# Change dataframe to df_div5ts

str2 = "https://dwr.state.co.us/Rest/GET/api/v2/structures/divrec/divrecyear/?wdid="
str3 = "%2C&apiKey=wAC6ZmzcPJ30dyy6nYu6jQmG7BBedcem"

for i in range(len(wdidlistB)):
    lstC = wdidlistB[i]
    lstCa = '%2C'.join([str(n) for n in lstC]) 
    
    url = str2 + lstCa + str3
    responseD = json.loads(requests.get(url).text)
    LD = responseD['ResultList']
    
    for n in range(len(LD)):
        row = pd.DataFrame([LD[n]])
        df_div5ts = df_div5ts.append(row)

print(len(df_div5ts))
df_div5ts.tail(3)

In [ ]:
# Merging wdid and division dataframes into one, using left-join.
df_div5ts = pd.merge(df_div5ts, dftemp, left_on='wdid', right_on='wdid', how='left')

In [ ]:
# Exporting to Finished File.
# Change dataframe to df_div5ts.
df_div5ts.to_excel('P_Div5WDIDTimeSeries.xlsx', index=False)  # The output

# Division 6

In [ ]:
# The Dvision 6 Time Series Dataframe
df_div6ts = pd.DataFrame()

In [ ]:
# Read in division csv file.
# Change input to Division 6.
fileInput = "Success/Division6.csv"
dftemp = pd.read_csv(fileInput)
print(len(dftemp))
dftemp.head(3)

In [ ]:
# Trim down division dataframe for divrectype = DivTotal, availableTimesteps = Year, and ciuCode = A (for active) values only.
# Fix wdid values that are less then 7 chars long.  Covert to string.
dftemp = dftemp[dftemp.divrectype == 'DivTotal']
dftemp = dftemp[dftemp.availableTimesteps == 'Year']
dftemp = dftemp[dftemp.ciuCode == 'A']

# fix wdid values that are less then 7 chars long.  Covert to string.
def formatWDIDValue(colVal):
    n = str(colVal)
    if len(n) < 7:
        outString = "0" + n
    else:
        outString = n
    return outString
dftemp['wdid'] = dftemp.apply(lambda row: formatWDIDValue(row['wdid']), axis=1)

print(len(dftemp))
dftemp.head(3)

In [ ]:
# Converted trimed down dataframe wdid to list.
dftemp = dftemp.drop_duplicates(subset='wdid', keep="first")
wdidlist = dftemp['wdid'].tolist()
type(wdidlist)

In [ ]:
# Split list into catagories that are 100 long.
wdidlistB = [wdidlist[i:i + 100] for i in range(0, len(wdidlist), 100)]

In [ ]:
# Use list of WDIDs (from Divisoin data) as inputs, retreive time series data.
# Change dataframe to df_div6ts

str2 = "https://dwr.state.co.us/Rest/GET/api/v2/structures/divrec/divrecyear/?wdid="
str3 = "%2C&apiKey=wAC6ZmzcPJ30dyy6nYu6jQmG7BBedcem"

for i in range(len(wdidlistB)):
    lstC = wdidlistB[i]
    lstCa = '%2C'.join([str(n) for n in lstC]) 
    
    url = str2 + lstCa + str3
    responseD = json.loads(requests.get(url).text)
    LD = responseD['ResultList']
    
    for n in range(len(LD)):
        row = pd.DataFrame([LD[n]])
        df_div6ts = df_div6ts.append(row)

print(len(df_div6ts))
df_div6ts.tail(3)

In [ ]:
# Merging wdid and division dataframes into one, using left-join.
df_div6ts = pd.merge(df_div6ts, dftemp, left_on='wdid', right_on='wdid', how='left')

In [ ]:
# Exporting to Finished File.
# Change dataframe to df_div6ts.
df_div6ts.to_excel('P_Div6WDIDTimeSeries.xlsx', index=False)  # The output

# Division 7

In [ ]:
# The Dvision 7 Time Series Dataframe
df_div7ts = pd.DataFrame()

In [ ]:
# Read in division csv file.
# Change input to Division 7.
fileInput = "Success/Division7.csv"
dftemp = pd.read_csv(fileInput)
print(len(dftemp))
dftemp.head(3)

In [ ]:
# Trim down division dataframe for divrectype = DivTotal, availableTimesteps = Year, and ciuCode = A (for active) values only.
# Fix wdid values that are less then 7 chars long.  Covert to string.
dftemp = dftemp[dftemp.divrectype == 'DivTotal']
dftemp = dftemp[dftemp.availableTimesteps == 'Year']
dftemp = dftemp[dftemp.ciuCode == 'A']

# fix wdid values that are less then 7 chars long.  Covert to string.
def formatWDIDValue(colVal):
    n = str(colVal)
    if len(n) < 7:
        outString = "0" + n
    else:
        outString = n
    return outString
dftemp['wdid'] = dftemp.apply(lambda row: formatWDIDValue(row['wdid']), axis=1)

print(len(dftemp))
dftemp.head(3)

In [ ]:
# Converted trimed down dataframe wdid to list.
dftemp = dftemp.drop_duplicates(subset='wdid', keep="first")
wdidlist = dftemp['wdid'].tolist()
type(wdidlist)

In [ ]:
# Split list into catagories that are 100 long.
wdidlistB = [wdidlist[i:i + 100] for i in range(0, len(wdidlist), 100)]

In [ ]:
# Use list of WDIDs (from Divisoin data) as inputs, retreive time series data.
# Change dataframe to df_div7ts

str2 = "https://dwr.state.co.us/Rest/GET/api/v2/structures/divrec/divrecyear/?wdid="
str3 = "%2C&apiKey=wAC6ZmzcPJ30dyy6nYu6jQmG7BBedcem"

for i in range(len(wdidlistB)):
    lstC = wdidlistB[i]
    lstCa = '%2C'.join([str(n) for n in lstC]) 
    
    url = str2 + lstCa + str3
    responseD = json.loads(requests.get(url).text)
    LD = responseD['ResultList']
    
    for n in range(len(LD)):
        row = pd.DataFrame([LD[n]])
        df_div7ts = df_div7ts.append(row)

print(len(df_div7ts))
df_div7ts.tail(3)

In [ ]:
# Merging wdid and division dataframes into one, using left-join.
df_div7ts = pd.merge(df_div7ts, dftemp, left_on='wdid', right_on='wdid', how='left')

In [ ]:
# Exporting to Finished File.
# Change dataframe to df_div7ts.
df_div7ts.to_excel('P_Div7WDIDTimeSeries.xlsx', index=False)  # The output

# Concatenate

In [ ]:
# Concatenate
frames = [df_div1ts, df_div2ts, df_div3ts, df_div4ts, df_div5ts, df_div6ts, df_div7ts]
dfout = pd.concat(frames)

In [ ]:
print(len(dfout))
dfout

# WaDE Custom Elements (due to missing sate info)

In [ ]:
# Create WaterSourceTypeCV
    
def createWaterSourceTypeCV(valA):
    if "GROUNDWATER:" in valA:
        outString = "Groundwater"
    else:
        outString = "Surface Water"
        
    return outString

dfout['in_WaterSourceTypeCV'] = dfout.apply(lambda row: createWaterSourceTypeCV( row['waterSource']), axis=1)
dfout.head(1)

In [ ]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDECO_WS" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = dfout['waterSource']
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = dfout['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceName'] == A)& 
                                       (dfWaterSourceNativeID['in_WaterSourceTypeCV'] == B), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

dfout['in_WaterSourceNativeID'] = dfout.apply(lambda row: retrieveWaterSourceNativeID( row['waterSource'], row['in_WaterSourceTypeCV']), axis=1)
dfout.head(3)

In [ ]:
# Creating WaDE Custom site native ID for easy site identificaiion
# ----------------------------------------------------------------------------------------------------

# Create temp SiteNativeID dataframe of unique site.
def assignSiteUUID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDECO_S" + string1
    return outstring

dfSiteNativeID = pd.DataFrame()
dfSiteNativeID['in_Latitude'] = dfout['latdecdeg']
dfSiteNativeID['in_Longitude'] = dfout['longdecdeg']
dfSiteNativeID['in_SiteTypeCV'] = dfout['structureType']
dfSiteNativeID['in_SiteName'] = dfout['structureName']
dfSiteNativeID = dfSiteNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfSiteNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfSiteNativeID['in_SiteNativeID'] = dftemp.apply(lambda row: assignSiteUUID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom site native ID
def retrieveSiteNativeID(A, B, C, D):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        ml = dfSiteNativeID.loc[(dfSiteNativeID['in_Latitude'] == A) & 
                                (dfSiteNativeID['in_Longitude'] == B) &
                                (dfSiteNativeID['in_SiteTypeCV'] == C) &
                                (dfSiteNativeID['in_SiteName'] == D), 'in_SiteNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

dfout['in_SiteNativeID'] = dfout.apply(lambda row: retrieveSiteNativeID( row['latdecdeg'], row['longdecdeg'], row['structureType'], row['structureName']), axis=1)
dfout.head(3)

In [ ]:
# TimeframeStart & TimeframeEnd

dfout['inTimeframeStart'] = '01/01/' + dfout['dataMeasDate'].astype(str)
dfout['inTimeframeEnd'] = '12/31/' + dfout['dataMeasDate'].astype(str)
dfout

In [ ]:
# Exporting to Finished File.
# Change dataframe to df_div7ts.
dfout.to_csv('P_coOSMaster.csv', index=False)  # The output

## Temp Fix: VariableSpecificCV
- Date: 03/09/2022
- Need to include a update to VariableSpecificCV field, but quering the CO REST API services takes too long.  Will instead use a temp fix here where I add the changes to the completed processed input file.

In [3]:
fileInput = "P_coOSMaster.csv"
dfco = pd.read_csv(fileInput)
print(len(dfco))
dfco.head()

C:\Users\rjame\AppData\Local\Temp\ipykernel_22176\2149495432.py:2: DtypeWarning: Columns (18,22,24,26,27,29,32,34,42,50,51,55,56,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  dfco = pd.read_csv(fileInput)


427381


,wdid,waterClassNum,wcIdentifier_x,measInterval,measCount,dataMeasDate,dataValue,measUnits,obsCode,approvalStatus,modified_x,divrectype,availableTimesteps,waterclassNum,wcIdentifier_y,wdidAcctId,structureName,wdidAcctName,sourceCode,sourceDescr,fromWdid,fromWdidAcctId,fromStructureName,fromWdidAcctName,useCode,useDescr,typeCode,typeDescr,groupWdid,groupStructureName,toWdid,toWdidAcctId,toStructureName,toWdidAcctName,wcDescr,porStart,porEnd,porLastModified,division,waterDistrict,county,modified_y,availableTimesteps.1,waterSource,streamMile,structureType,gnisId,ciuCode,ciuCodeLong,subdistrict,designatedBasinName,managementDistrictName,pm,township,range,section,q10,q40,q160,coordsew,coordsewDir,coordsns,coordsnsDir,utmX,utmY,latdecdeg,longdecdeg,in_WaterSourceNativeID,in_SiteNativeID,inTimeframeStart,inTimeframeEnd,in_WaterSourceTypeCV,in_VariableSpecificCV
0,100509,54282,0100509 S:1 F: U:1 T: G: To:,Monthly,1,1974,971.9200,ACFT,*,Approved,2001-08-16T17:46:00,WaterClass,Year,54716,0100509 S:1 F: U:1 T:4 G: To:,NaN,CORONA RANCH DITCH,NaN,1,Natural Stream flow,NaN,NaN,NaN,NaN,1,Irrigation,4,Alternate Point of Diversion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985-10-31T00:00:00,1985-10-31T00:00:00,2001-08-16T17:46:00,1,1,WELD,2011-06-02T08:52:50.99,Year,SOUTH PLATTE RIVER,154.27,DITCH,201759.0,A,Active Structure with contemporary diversion r...,NaN,NaN,NaN,S,4.0 N,63.0 W,13.0,NE,NE,NE,NaN,NaN,NaN,NaN,552953.0,4463159.0,40.317282,-104.376767,WaDECO_WS1,WaDECO_S1,01/01/1974,12/31/1974,Surface Water,Stream Gage_Annual_DivTotal_Surface Water
1,100509,54282,0100509 S:1 F: U:1 T: G: To:,Annual,1,1980,162.0000,ACFT,*,Approved,2001-08-16T17:46:00,WaterClass,Year,54716,0100509 S:1 F: U:1 T:4 G: To:,NaN,CORONA RANCH DITCH,NaN,1,Natural Stream flow,NaN,NaN,NaN,NaN,1,Irrigation,4,Alternate Point of Diversion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985-10-31T00:00:00,1985-10-31T00:00:00,2001-08-16T17:46:00,1,1,WELD,2011-06-02T08:52:50.99,Year,SOUTH PLATTE RIVER,154.27,DITCH,201759.0,A,Active Structure with contemporary diversion r...,NaN,NaN,NaN,S,4.0 N,63.0 W,13.0,NE,NE,NE,NaN,NaN,NaN,NaN,552953.0,4463159.0,40.317282,-104.376767,WaDECO_WS1,WaDECO_S1,01/01/1980,12/31/1980,Surface Water,Stream Gage_Annual_DivTotal_Surface Water
2,100509,54282,0100509 S:1 F: U:1 T: G: To:,Daily,72,1986,1246.2132,ACFT,*,Approved,2001-08-14T18:04:00,WaterClass,Year,54716,0100509 S:1 F: U:1 T:4 G: To:,NaN,CORONA RANCH DITCH,NaN,1,Natural Stream flow,NaN,NaN,NaN,NaN,1,Irrigation,4,Alternate Point of Diversion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985-10-31T00:00:00,1985-10-31T00:00:00,2001-08-16T17:46:00,1,1,WELD,2011-06-02T08:52:50.99,Year,SOUTH PLATTE RIVER,154.27,DITCH,201759.0,A,Active Structure with contemporary diversion r...,NaN,NaN,NaN,S,4.0 N,63.0 W,13.0,NE,NE,NE,NaN,NaN,NaN,NaN,552953.0,4463159.0,40.317282,-104.376767,WaDECO_WS1,WaDECO_S1,01/01/1986,12/31/1986,Surface Water,Stream Gage_Annual_DivTotal_Surface Water
3,100509,54282,0100509 S:1 F: U:1 T: G: To:,Daily,146,1987,2417.8072,ACFT,*,Approved,2001-08-14T18:04:00,WaterClass,Year,54716,0100509 S:1 F: U:1 T:4 G: To:,NaN,CORONA RANCH DITCH,NaN,1,Natural Stream flow,NaN,NaN,NaN,NaN,1,Irrigation,4,Alternate Point of Diversion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985-10-31T00:00:00,1985-10-31T00:00:00,2001-08-16T17:46:00,1,1,WELD,2011-06-02T08:52:50.99,Year,SOUTH PLATTE RIVER,154.27,DITCH,201759.0,A,Active Structure with contemporary diversion r...,NaN,NaN,NaN,S,4.0 N,63.0 W,13.0,NE,NE,NE,NaN,NaN,NaN,NaN,552953.0,4463159.0,40.317282,-104.376767,WaDECO_WS1,WaDECO_S1,01/01/1987,12/31/1987,Surface Water,Stream Gage_Annual_DivTotal_Surface Water
4,100509,54282,0100509 S:1 F: U:1 T: G: To:,Daily,200,1988,3496.3155,ACFT,*,Approved,2001-08-14T18:04:00,WaterClass,Year,54716,0100509 S:1 F: U:1 T:4 G: To:,NaN,CORONA RANCH DITCH,NaN,1,Natural Stream flow,NaN,NaN,NaN,NaN,1,Irrigation,4,Alternate Point of Diversion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985-10-31T00:00:00,1985-10-31T00:00:00,2001-08-16T17:46:00,1,1,WELD,2011-06-02T08:52:50.99,Year,SOUTH PLATTE RIVER,154

In [ ]:
def creatVarspec(WST):
    WST = WST.strip()
    outString = "Stream Gage_Annual_DivTotal_" + WST
    return outString

dfco['in_VariableSpecificCV'] = dfco.apply(lambda row: creatVarspec(row['in_WaterSourceTypeCV']), axis=1)
dfco['in_VariableSpecificCV'].unique()

In [ ]:
# Exporting to Finished File.
# Change dataframe to df_div7ts.
dfco.to_csv('P_coOSMaster.csv', index=False)  # The output

In [8]:
dfco['useDescr'].unique()

array(['Irrigation', 'Recreation', 'Commercial', 'Stock', 'Municipal',
       'Domestic', 'Recharge', 'Storage', 'Augmentation',
       'Quantification of amount', 'Industrial', 'Evaporation',
       'Change of Use Return Flows ', 'Power Generation', 'Other',
       'All Beneficial Uses', 'Household use only', 'Fishery',
       'Sub-basin export', 'Wildlife', 'Transbasin export', 'Fire',
       'Snow Making', 'Minimum stream flow/lake level',
       'Export from State'], dtype=object)